In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/spam.csv',encoding ='latin1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
data = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

In [ ]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
data.isnull().sum()

v1    0
v2    0
dtype: int64

In [ ]:
data.duplicated().sum()

403

In [ ]:
data = data.drop_duplicates()
data.duplicated().sum()

0

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5169 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5169 non-null   object
 1   v2      5169 non-null   object
dtypes: object(2)
memory usage: 121.1+ KB


In [ ]:
data.rename(columns={"v1":"label","v2":"text"},inplace=True)

In [ ]:
data['label'].unique()

array(['ham', 'spam'], dtype=object)

In [ ]:
data["label"].value_counts()

label
ham     4516
spam     653
Name: count, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])
data['label']

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: label, Length: 5169, dtype: int64

In [ ]:
data.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
from bs4 import BeautifulSoup
import re
import string
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Remove special characters and digits
    text = re.sub(r"[^a-zA-Z]", " ", text)

    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Remove emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Convert to lowercase
    text = text.lower()

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('punkt')
data["text"]=data["text"].apply(clean_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
<ipython-input-17-f19c5375cc1a>:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [ ]:
data['number_character']= data['text'].apply(len)
data['number_of_words'] = data['text'].apply(lambda x: len(x.split()))
data['number_sentence'] = data['text'].apply(lambda x: len(nltk.sent_tokenize(x)))

In [ ]:
data.head()

,label,text,number_character,number_of_words,number_sentence
0,0,go jurong point crazy available bugis n great ...,82,16,1
1,0,ok lar joking wif u oni,23,6,1
2,1,free entry wkly comp win fa cup final tkts st ...,101,21,1
3,0,u dun say early hor u c already say,35,9,1
4,0,nah think goes usf lives around though,38,7,1


In [ ]:
data[["number_character","number_of_words","number_sentence"]].describe()


,number_character,number_of_words,number_sentence
count,5169.000000,5169.000000,5169.000000
mean,48.267557,8.691236,0.998646
std,36.666598,6.350770,0.036778
min,0.000000,0.000000,0.000000
25%,21.000000,4.000000,1.000000
50%,37.000000,7.000000,1.000000
75%,71.000000,12.000000,1.000000
max,447.000000,73.000000,1.000000


In [ ]:
data[data["label"]==0][["number_character","number_of_words","number_sentence"]].describe()

,number_character,number_of_words,number_sentence
count,4516.000000,4516.000000,4516.000000
mean,42.524579,7.717671,0.998450
std,34.595643,5.960242,0.039344
min,0.000000,0.000000,0.000000
25%,19.000000,4.000000,1.000000
50%,32.000000,6.000000,1.000000
75%,56.000000,10.000000,1.000000
max,447.000000,73.000000,1.000000


In [ ]:
data[data["label"]==1][["number_character","number_of_words","number_sentence"]].describe()

,number_character,number_of_words,number_sentence
count,653.000000,653.000000,653.0
mean,87.984686,15.424196,1.0
std,23.667749,4.660482,0.0
min,4.000000,1.000000,1.0
25%,77.000000,13.000000,1.0
50%,92.000000,16.000000,1.0
75%,104.000000,19.000000,1.0
max,142.000000,26.000000,1.0


## **TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
x = tf.fit_transform(data["text"])
y = data["label"]

## **Splitting of Data**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

## **Support Vector Machine (SVM)**

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')
svm.fit(x_train,y_train)
pred = svm.predict(x_test)

In [ ]:
#Evaluate Accuracy
accuracy = accuracy_score(y_test, pred)
print(f"Accuracy:",accuracy)

Accuracy: 0.9816247582205029
